In [34]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [35]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

2.2.3
3.4.2


In [36]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time



In [37]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
"""
    Parameters
    ----------
    problem_formulation_id : int {0, ..., 5}
                             problem formulations differ with respect to the objectives
                             0: Total cost, and casualties
                             1: Expected damages, costs, and casualties
                             2: expected damages, dike investment costs, rfr costs, evacuation cost, and casualties
                             3: costs and casualties disaggregated over dike rings, and room for the river and evacuation costs
                             4: Expected damages, dike investment cost and casualties disaggregated over dike rings and room for the river and evacuation costs
                             5: disaggregate over time and space

    Notes
    -----
    problem formulations 4 and 5 rely on ArrayOutcomes and thus cannot straightforwardly
    be used in optimizations

    """
problem_formulation = 3
dike_model, planning_steps = get_model_for_problem_formulation(problem_formulation)

In [38]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfai

In [39]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('0_RfR 1', 0, 1, resolution=None, default=None, variable_name=['0_RfR 1'], pff=False)
IntegerParameter('0_RfR 2', 0, 1, resolution=None, default=None, variable_name=['0_RfR 2'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('1_RfR 1', 0, 1, resolution=None, default=None, variable_name=['1_RfR 1'], pff=False)
IntegerParameter('1_RfR 2', 0, 1, resolution=None, default=None, variable_name=['1_RfR 2'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('2_RfR 1', 0, 1, resolution=None, default=None, variable_name=['2_RfR 1'], pff=False)
IntegerParameter('2_RfR 2', 0, 1, resolution=None, default=None, variable_name=['2_RfR 2'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable

In [40]:
# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1 Total Costs', variable_name=('A.1_Expected Annual Damage', 'A.1_Dike Investment Costs'), function=<function sum_over at 0x000001D4BF252950>)
ScalarOutcome('A.1_Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths',), function=<function sum_over at 0x000001D4BF252950>)
ScalarOutcome('A.2 Total Costs', variable_name=('A.2_Expected Annual Damage', 'A.2_Dike Investment Costs'), function=<function sum_over at 0x000001D4BF252950>)
ScalarOutcome('A.2_Expected Number of Deaths', variable_name=('A.2_Expected Number of Deaths',), function=<function sum_over at 0x000001D4BF252950>)
ScalarOutcome('A.3 Total Costs', variable_name=('A.3_Expected Annual Damage', 'A.3_Dike Investment Costs'), function=<function sum_over at 0x000001D4BF252950>)
ScalarOutcome('A.3_Expected Number of Deaths', variable_name=('A.3_Expected Number of Deaths',), function=<function sum_over at 0x000001D4BF252950>)
ScalarOutcome('A.4 Total Costs', variable_name=('A.4_Expected Annual Dama

In [41]:
# running the model through EMA workbench
with SequentialEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=2, policies=4)

[MainProcess/INFO] performing 2 scenarios * 4 policies * 1 model(s) = 8 experiments
100%|████████████████████████████████████████████| 8/8 [00:01<00:00,  4.01it/s]
[MainProcess/INFO] experiments finished


In [42]:
# observing the simulation runs
experiments, outcomes = results
print(outcomes.keys())
experiments

dict_keys(['A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs', 'A.2_Expected Number of Deaths', 'A.3 Total Costs', 'A.3_Expected Number of Deaths', 'A.4 Total Costs', 'A.4_Expected Number of Deaths', 'A.5 Total Costs', 'A.5_Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs'])


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,A.4_Bmax,A.4_Brate,A.4_pfail,A.5_Bmax,A.5_Brate,A.5_pfail,discount rate 0,discount rate 1,discount rate 2,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,3_RfR 1,3_RfR 2,4_RfR 0,4_RfR 1,4_RfR 2,A.1_DikeIncrease 0,A.1_DikeIncrease 1,A.1_DikeIncrease 2,A.2_DikeIncrease 0,A.2_DikeIncrease 1,A.2_DikeIncrease 2,A.3_DikeIncrease 0,A.3_DikeIncrease 1,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,107,317.933441,1.5,0.202942,71.503414,1.0,0.060954,338.005554,10.0,0.057438,333.491940,1.0,0.740807,196.767723,1.0,0.506409,2.5,2.5,3.5,1,0,0,1,1,0,0,1,0,0,1,1,1,0,0,4,10,4,10,1,1,8,4,1,8,5,10,4,7,1,4,90,86,dikesnet
1,38,78.707788,10.0,0.594267,300.170175,10.0,0.699875,107.386488,1.5,0.655918,60.591059,10.0,0.196234,59.836257,10.0,0.109788,4.5,3.5,2.5,1,0,0,1,1,0,0,1,0,0,1,1,1,0,0,4,10,4,10,1,1,8,4,1,8,5,10,4,7,1,4,91,86,dikesnet
2,107,317.933441,1.5,0.202942,71.503414,1.0,0.060954,338.005554,10.0,0.057438,333.491940,1.0,0.740807,196.767723,1.0,0.506409,2.5,2.5,3.5,0,0,0,0,1,1,1,0,1,0,0,1,0,1,1,8,8,8,0,5,4,10,2,4,4,2,6,0,3,7,3,90,87,dikesnet
3,38,78.707788,10.0,0.594267,300.170175,10.0,0.699875,107.386488,1.5,0.655918,60.591059,10.0,0.196234,59.836257,10.0,0.109788,4.5,3.5,2.5,0,0,0,0,1,1,1,0,1,0,0,1,0,1,1,8,8,8,0,5,4,10,2,4,4,2,6,0,3,7,3,91,87,dikesnet
4,107,317.933441,1.5,0.202942,71.503414,1.0,0.060954,338.005554,10.0,0.057438,333.491940,1.0,0.740807,196.767723,1.0,0.506409,2.5,2.5,3.5,0,1,1,0,0,0,0,0,1,1,0,0,0,0,1,7,4,8,5,10,8,1,8,8,1,9,4,6,10,10,0,90,88,dikesnet
5,38,78.707788,10.0,0.594267,300.170175,10.0,0.699875,107.386488,1.5,0.655918,60.591059,10.0,0.196234,59.836257,10.0,0.109788,4.5,3.5,2.5,0,1,1,0,0,0,0,0,1,1,0,0,0,0,1,7,4,8,5,10,8,1,8,8,1,9,4,6,10,10,0,91,88,dikesnet
6,107,317.933441,1.5,0.202942,71.503414,1.0,0.060954,338.005554,10.0,0.057438,333.491940,1.0,0.740807,196.767723,1.0,0.506409,2.5,2.5,3.5,1,1,1,1,0,1,1,1,0,1,1,0,1,1,0,1,1,1,6,3,8,4,8,8,6,0,2,10,1,3,1,90,89,dikesnet
7,38,78.707788,10.0,0.594267,300.170175,10.0,0.699875,107.386488,1.5,0.655918,60.591059,10.0,0.196234,59.836257,10.0,0.109788,4.5,3.5,2.5,1,1,1,1,0,1,1,1,0,1,1,0,1,1,0,1,1,1,6,3,8,4,8,8,6,0,2,10,1,3,1,91,89,dikesnet


In [43]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,2.332236e+08,0.0,2.083899e+08,0.000000,1.022111e+08,0.000000,6.214101e+07,0.000000,1.201606e+08,0.000000,1.049400e+09,0.000000
1,2.332236e+08,0.0,2.083899e+08,0.000000,1.022111e+08,0.000000,6.214101e+07,0.000000,1.216755e+08,0.000247,1.049400e+09,166.502239
2,3.067442e+08,0.0,2.824713e+08,0.015910,1.147508e+08,0.000000,3.347975e+07,0.000000,7.923200e+07,0.000000,1.130400e+09,8887.598090
3,3.067442e+08,0.0,1.254217e+08,0.000495,1.147508e+08,0.000000,3.347975e+07,0.000000,1.536915e+08,0.011326,1.130400e+09,7204.254416
4,2.390911e+08,0.0,3.204636e+08,0.008174,4.139893e+08,0.483272,3.865174e+07,0.000000,2.110367e+08,0.000000,5.772000e+08,0.000000
5,2.390911e+08,0.0,3.116832e+08,0.000000,1.119265e+08,0.007060,4.181251e+07,0.002239,2.110367e+08,0.000000,5.772000e+08,0.000000
6,1.014745e+08,0.0,2.366512e+08,0.000709,1.331022e+08,0.005104,2.155622e+07,0.000000,1.371356e+08,0.000000,1.505400e+09,229.150409
7,1.014745e+08,0.0,2.345505e+08,0.000000,1.248529e+08,0.000000,2.155622e+07,0.000000,1.371356e+08,0.000000,1.505400e+09,0.000000


In [44]:
from ema_workbench.em_framework.samplers import sample_levers

# defining specific policies
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations
# policy 4 doe niks
'''
A1 = Doesburg Upstream
A2 = Cortenoever Upmidstream
A3 = Zutphen Midstream
A4 = Gorssel Downmidstream
A5 = Deventer Downstream
0_RFR = Project Olburgen
1_RFR = Project Havikerwaard
2_RFR = project Tichelbeekse
3_RFR = Project Welsummer
4_RFR = Obstakelsverwijderen
dan 0 of 1 of 2 is de timestep waarin het wordt geddaan
'''

n_policies = 100

def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}

# Sample 1000 valid random lever combinations for dike_model
policy_dicts = sample_levers(dike_model, n_samples=500)

# Convert each to a Policy object
policies = [Policy(f"policy {i}", **d) for i, d in enumerate(policy_dicts)]

# policies = [
#     Policy(
#         "policy 1",
#         **dict(
#             get_do_nothing_dict(),
#             **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "2_RfR 0": 1, "2_RfR 1": 1, "2_RfR 2": 1, "A.3_DikeIncrease 0": 5, "A.3_DikeIncrease 1": 5}
#         )
#     ),
#     Policy(
#         "policy 2",
#         **dict(
#             get_do_nothing_dict(),
#             **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "2_RfR 0": 1, "2_RfR 1": 1, "2_RfR 2": 1, "A.3_DikeIncrease 0": 5}
#         )),
    #     Policy(
    #     "policy x",
    #     **dict(
    #         get_do_nothing_dict(),
    #         **{"4_RfR 0": 1}
    #     )
    # ),
    
    # Policy(
    #     "policy 2",
    #     **dict(
    #         get_do_nothing_dict(),
    #         **{"4_RfR 0": 1, "4_RfR 1": 1, "4_RfR 2": 1, "A.5_DikeIncrease 0": 5}
    #     )
    # ),
    # Policy(
    #     "policy 3",
    #     **dict(
    #         get_do_nothing_dict(),
    #         **{"1_RfR 0": 1, "2_RfR 1": 1, "3_RfR 2": 1, "A.3_DikeIncrease 0": 5}
    #     )
    # ),
    # Policy(
    #     "policy 4",
    #     **dict(
    #         get_do_nothing_dict(),
            
    #     )
    # ),
# ]

In [45]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 20
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies= policies)

[MainProcess/INFO] pool started with 16 workers
[MainProcess/INFO] performing 20 scenarios * 500 policies * 1 model(s) = 10000 experiments
100%|████████████████████████████████████| 10000/10000 [04:21<00:00, 38.19it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [46]:
experiments, outcomes = results

In [71]:
from sklearn.preprocessing import MinMaxScaler

# only works because we have scalar outcomes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df_experiments = pd.DataFrame(experiments)
df_outcomes = pd.DataFrame(outcomes)

# aggregating outcomes
cost_keys = [k for k in outcomes if 'Total Costs' in k]
df_outcomes['Total costs'] = np.sum([df_outcomes[k] for k in cost_keys], axis=0)

cost_keys = [k for k in outcomes if 'Number of Deaths' in k]
df_outcomes['Number of Deaths'] = np.sum([df_outcomes[k] for k in cost_keys], axis=0)

mean_outcomes = df_outcomes.copy()
mean_outcomes['policy'] = df_experiments['policy']

policy_means = mean_outcomes.groupby('policy')[['Total costs', 'Number of Deaths']].mean()

# Normalize deaths and costs to [0, 1] range
scaler = MinMaxScaler()
scaled = scaler.fit_transform(policy_means[['Total costs', 'Number of Deaths']])

# Put scaled values back into the DataFrame
policy_means['costs_scaled'] = scaled[:, 0]
policy_means['deaths_scaled'] = scaled[:, 1]

# Now create a composite score using equal weighting
policy_means['score'] = policy_means['costs_scaled'] + policy_means['deaths_scaled']
#policy_means['score'] = 1.0 * policy_means['costs_scaled']

# 4. Select top 100 policies
top_policies = policy_means.sort_values('score').head(100)

# 5. Get corresponding lever settings
# Assume all rows with same policy name have same levers (since policy is fixed per group)
top_policy_names = top_policies.index.tolist()

# Get the first matching row per top policy
policy_settings = df_experiments[df_experiments['policy'].isin(top_policy_names)]
policy_settings = policy_settings.drop_duplicates(subset='policy')
policy_settings = policy_settings.set_index('policy').loc[top_policy_names]  # same order as top_policies

# 6. Combine and display
top_summary = pd.concat([top_policies, policy_settings], axis=1)

top_summary

C:\Users\juliu\AppData\Local\Temp\ipykernel_240\3520584581.py:20: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  policy_means = mean_outcomes.groupby('policy')[['Total costs', 'Number of Deaths']].mean()


,Total costs,Number of Deaths,costs_scaled,deaths_scaled,score,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,A.4_Bmax,A.4_Brate,A.4_pfail,A.5_Bmax,A.5_Brate,A.5_pfail,discount rate 0,discount rate 1,discount rate 2,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,3_RfR 1,3_RfR 2,4_RfR 0,4_RfR 1,4_RfR 2,EWS_DaysToThreat,A.1_DikeIncrease 0,A.1_DikeIncrease 1,A.1_DikeIncrease 2,A.2_DikeIncrease 0,A.2_DikeIncrease 1,A.2_DikeIncrease 2,A.3_DikeIncrease 0,A.3_DikeIncrease 1,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,scenario,model
policy,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
policy 220,9.784982e+08,0.002912,0.016159,0.007638,0.023797,101,105.6418,1.5,0.312258,35.90999,10.0,0.739429,127.942172,1.0,0.602672,243.852244,10.0,0.041142,98.677718,10.0,0.650582,3.5,1.5,2.5,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,4,7,1,2,1,3,3,9,9,7,7,0,3,7,3,7,592,dikesnet
policy 371,9.844273e+08,0.002362,0.019355,0.006197,0.025552,101,105.6418,1.5,0.312258,35.90999,10.0,0.739429,127.942172,1.0,0.602672,243.852244,10.0,0.041142,98.677718,10.0,0.650582,3.5,1.5,2.5,1,0,0,0,0,0,1,1,1,1,0,0,0,0,0,2,4,9,0,5,6,10,2,5,3,0,7,4,3,6,6,592,dikesnet
policy 409,9.517100e+08,0.012810,0.001717,0.033602,0.035320,101,105.6418,1.5,0.312258,35.90999,10.0,0.739429,127.942172,1.0,0.602672,243.852244,10.0,0.041142,98.677718,10.0,0.650582,3.5,1.5,2.5,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,5,1,7,1,8,9,4,9,7,3,3,3,10,10,0,592,dikesnet
policy 387,9.976284e+08,0.003727,0.026472,0.009778,0.036250,101,105.6418,1.5,0.312258,35.90999,10.0,0.739429,127.942172,1.0,0.602672,243.852244,10.0,0.041142,98.677718,10.0,0.650582,3.5,1.5,2.5,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,4,2,9,1,10,3,1,4,7,5,0,10,1,1,6,2,592,dikesnet
policy 133,1.067435e+09,0.002413,0.064106,0.006330,0.070436,101,105.6418,1.5,0.312258,35.90999,10.0,0.739429,127.942172,1.0,0.602672,243.852244,10.0,0.041142,98.677718,10.0,0.650582,3.5,1.5,2.5,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,2,5,0,2,4,7,3,2,10,7,10,9,6,10,4,5,592,dikesnet
policy 14,1.097612e+09,0.007337,0.080374,0.019246,0.099620,101,105.6418,1.5,0.312258,35.90999,10.0,0.739429,127.942172,1.0,0.602672,243.852244,10.0,0.041142,98.677718,10.0,0.650582,3.5,1.5,2.5,1,0,0,0,0,0,1,1,0,0,1,0,0,0,1,2,1,2,2,4,2,2,6,9,7,2,2,4,6,2,0,592,dikesnet
policy 480,1.114394e+09,0.006027,0.089422,0.015809,0.105231,101,105.6418,1.5,0.312258,35.90999,10.0,0.739429,127.942172,1.0,0.602672,243.852244,10.0,0.041142,98.677718,10.0,0.650582,3.5,1.5,2.5,1,1,0,0,0,0,1,1,0,1,0,0,0,0,1,3,3,7,0,2,0,1,2,7,10,5,6,4,2,2,5,592,dikesnet
policy 121,1.119563e+09,0.009164,0.092209,0.024039,0.116247,101,105.6418,1.5,0.312258,35.90999,10.0,0.739429,127.942172,1.0,0.602672,243.852244,10.0,0.041142,98.677718,10.0,0.650582,3.5,1.5,2.5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,4,5,5,4,4,8,3,3,10,6,8,9,6,9,3,6,592,dikesnet
policy 214,1.164419e+09,0.001352,0.116391,0.003546,0.119937,101,105.6418,1.5,0.312258,35.90999,10.0,0.739429,127.942172,1.0,0.602672,243.852244,10.0,0.041142,98.677718,10.0,0.650582,3.5,1.5,2.5,0,0,1,1,0,0,0,1,1,0,1,0,0,1,0,3,2,3,0,6,4,0,4,3,7,0,6,5,4,4,0,592,dikesnet


In [74]:
# 1. Get top 100 and top 500 policy names
policy_means_sorted = policy_means.sort_values('score')
top_100_names = policy_means_sorted.head(100).index
top_500_names = policy_means_sorted.head(500).index

# 2. Compute average deaths for each group
avg_deaths_top_100 = policy_means.loc[top_100_names, 'Number of Deaths'].mean()
avg_deaths_top_500 = policy_means.loc[top_500_names, 'Number of Deaths'].mean()

# 3. Print results
print(f"Average deaths in top 100 policies: {avg_deaths_top_100:.4f}")
print(f"Average deaths in top 500 policies: {avg_deaths_top_500:.4f}")

Average deaths in top 100 policies: 0.0096
Average deaths in top 500 policies: 0.0145


In [72]:
from sklearn.preprocessing import MinMaxScaler

# 1. Get top 100 policies and their scores
top_policies = policy_means.sort_values('score').head(100)
top_policy_names = top_policies.index.tolist()
top_scores = top_policies['score']

# 2. Get lever settings for those top policies
top_experiments = df_experiments[df_experiments['policy'].isin(top_policy_names)]
top_experiments = top_experiments.drop_duplicates(subset='policy')

# 3. Align lever data and score weights
top_experiments = top_experiments.set_index('policy').loc[top_policy_names]  # ensure order matches
lever_names = [lever.name for lever in dike_model.levers]
lever_data = top_experiments[lever_names].astype(float)

# 4. Standardize lever values to [0, 1] across top 100
scaler = MinMaxScaler()
lever_data_scaled = pd.DataFrame(
    scaler.fit_transform(lever_data),
    columns=lever_data.columns,
    index=lever_data.index
)

# 5. Weight by inverse score (lower score = better = higher weight)
weights = 1 / top_scores  # inverse score
weights = weights / weights.sum()  # normalize to sum to 1

# 6. Compute weighted average importance
lever_importance = (lever_data_scaled.T @ weights).sort_values(ascending=False)

# 7. Convert to DataFrame for display
importance_df = lever_importance.reset_index()
importance_df.columns = ['Lever', 'Weighted Importance (Top 100 Policies)']
importance_df

,Lever,Weighted Importance (Top 100 Policies)
0,0_RfR 0,0.610259
1,2_RfR 1,0.590646
2,EWS_DaysToThreat,0.580045
3,A.3_DikeIncrease 1,0.575588
4,A.3_DikeIncrease 0,0.541603
5,A.5_DikeIncrease 0,0.510066
6,A.3_DikeIncrease 2,0.494411
7,A.4_DikeIncrease 1,0.490770
8,2_RfR 0,0.479546
9,A.2_DikeIncrease 1,0.476164


In [69]:
# 1. Get mask for policy 220
mask = experiments['policy'] == 'policy 409'

# 2. Convert outcomes for this policy to a DataFrame
policy_220_outcomes = pd.DataFrame({k: v[mask] for k, v in outcomes.items()})

# 3. Identify relevant outcome keys
cost_keys = [k for k in policy_220_outcomes.columns if 'Total Costs' in k]
death_keys = [k for k in policy_220_outcomes.columns if 'Number of Deaths' in k]

# 4. Compute the mean across those columns (per row)
policy_220_outcomes['Total costs (mean)'] = policy_220_outcomes[cost_keys].sum(axis=1)
policy_220_outcomes['Number of Deaths (mean)'] = policy_220_outcomes[death_keys].sum(axis=1)

# 5. Optionally include scenario info
scenarios = experiments[mask]['scenario'].values
policy_220_outcomes['Scenario'] = scenarios

# 6. Reorder columns to show the new ones first
cols = ['Scenario', 'Total costs (mean)', 'Number of Deaths (mean)'] + [col for col in policy_220_outcomes.columns if col not in ['Scenario', 'Total costs (mean)', 'Number of Deaths (mean)']]
policy_220_outcomes = policy_220_outcomes[cols]
policy_220_outcomes


,Scenario,Total costs (mean),Number of Deaths (mean),A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,592,9.461394e+08,0.007159,1.760069e+08,0.0,2.476902e+08,0.001562,1.258204e+08,0.000000,5.713644e+07,0.005597,1.395855e+08,0.0,199900000.0,1267.220100
1,593,9.193891e+08,0.001545,1.760069e+08,0.0,2.499315e+08,0.001545,1.258204e+08,0.000000,2.814489e+07,0.000000,1.395855e+08,0.0,199900000.0,114.813031
2,594,9.297296e+08,0.005345,1.760069e+08,0.0,2.600052e+08,0.005293,1.258204e+08,0.000000,2.841167e+07,0.000052,1.395855e+08,0.0,199900000.0,416.632054
3,595,9.173181e+08,0.001809,1.760069e+08,0.0,2.470301e+08,0.001555,1.258204e+08,0.000000,2.897520e+07,0.000254,1.395855e+08,0.0,199900000.0,153.222617
4,596,9.135259e+08,0.000000,1.760069e+08,0.0,2.440682e+08,0.000000,1.258204e+08,0.000000,2.814489e+07,0.000000,1.395855e+08,0.0,199900000.0,0.000000
5,597,9.554331e+08,0.039293,1.760069e+08,0.0,2.443055e+08,0.000127,1.674903e+08,0.039166,2.814489e+07,0.000000,1.395855e+08,0.0,199900000.0,1390.245601
6,598,9.192022e+08,0.002404,1.760069e+08,0.0,2.497445e+08,0.002404,1.258204e+08,0.000000,2.814489e+07,0.000000,1.395855e+08,0.0,199900000.0,181.703892
7,599,9.407258e+08,0.011064,1.760069e+08,0.0,2.710864e+08,0.011022,1.258204e+08,0.000000,2.832661e+07,0.000042,1.395855e+08,0.0,199900000.0,882.348291
8,600,9.182144e+08,0.002439,1.760069e+08,0.0,2.487568e+08,0.002439,1.258204e+08,0.000000,2.814489e+07,0.000000,1.395855e+08,0.0,199900000.0,182.667497
9,601,9.303300e+08,0.006978,1.760069e+08,0.0,2.563923e+08,0.004145,1.303004e+08,0.002833,2.814489e+07,0.000000,1.395855e+08,0.0,199900000.0,407.833474


In [ ]:
# Filter all rows where policy == 'policy 220'
policy_220_experiments = experiments[experiments['policy'] == 'policy 155']
policy_220_outcomes = {k: v[experiments['policy'] == 'policy 155'] for k, v in outcomes.items()}

# Combine experiments and outcomes into a single DataFrame for display
import pandas as pd

df_outcomes = pd.DataFrame(policy_220_outcomes)
df_policy_220 = pd.concat([policy_220_experiments.reset_index(drop=True), df_outcomes.reset_index(drop=True)], axis=1)
df_policy_220
# 1. Get the mask for policy 220
mask = experiments['policy'] == 'policy 220'

# 2. Extract the corresponding Total costs
total_costs_220 = outcomes['Total costs'][mask]

# 3. Put in a DataFrame with scenario index
policy_220_total_costs = pd.DataFrame({
    'Scenario': experiments[mask]['scenario'].values,
    'Total costs': total_costs_220
})

,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,A.4_Bmax,A.4_Brate,A.4_pfail,A.5_Bmax,A.5_Brate,A.5_pfail,discount rate 0,discount rate 1,discount rate 2,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,3_RfR 1,3_RfR 2,4_RfR 0,4_RfR 1,4_RfR 2,EWS_DaysToThreat,A.1_DikeIncrease 0,A.1_DikeIncrease 1,A.1_DikeIncrease 2,A.2_DikeIncrease 0,A.2_DikeIncrease 1,A.2_DikeIncrease 2,A.3_DikeIncrease 0,A.3_DikeIncrease 1,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,scenario,policy,model,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,101,105.641800,1.5,0.312258,35.909990,10.0,0.739429,127.942172,1.0,0.602672,243.852244,10.0,0.041142,98.677718,10.0,0.650582,3.5,1.5,2.5,0,0,0,0,1,0,1,1,0,0,1,1,1,0,0,4,8,8,6,0,6,2,1,6,9,1,2,10,9,1,8,592,policy 155,dikesnet,2.800699e+08,0.0,1.219673e+08,0.000498,1.041480e+08,0.000000,7.064072e+07,0.001807,1.548091e+08,0.0,777700000.0,3500.267218
1,46,187.933564,1.5,0.476218,166.552517,1.0,0.638196,91.865315,10.0,0.353449,215.072538,1.0,0.517173,125.326136,1.0,0.230928,1.5,3.5,2.5,0,0,0,0,1,0,1,1,0,0,1,1,1,0,0,4,8,8,6,0,6,2,1,6,9,1,2,10,9,1,8,593,policy 155,dikesnet,2.800699e+08,0.0,1.241571e+08,0.000493,1.092171e+08,0.000810,3.657132e+07,0.000006,1.548091e+08,0.0,777700000.0,478.373828
2,63,334.659292,1.0,0.094333,69.530989,10.0,0.364151,163.735651,1.5,0.767751,321.096761,1.0,0.287006,163.180410,10.0,0.399296,2.5,1.5,2.5,0,0,0,0,1,0,1,1,0,0,1,1,1,0,0,4,8,8,6,0,6,2,1,6,9,1,2,10,9,1,8,594,policy 155,dikesnet,2.800699e+08,0.0,1.339481e+08,0.001679,1.041480e+08,0.000000,3.711764e+07,0.000035,1.548091e+08,0.0,777700000.0,1017.592257
3,86,149.408827,1.0,0.277432,281.282546,1.0,0.669231,145.156760,10.0,0.339955,223.484975,10.0,0.328546,69.828802,1.5,0.574218,4.5,2.5,1.5,0,0,0,0,1,0,1,1,0,0,1,1,1,0,0,4,8,8,6,0,6,2,1,6,9,1,2,10,9,1,8,595,policy 155,dikesnet,2.800699e+08,0.0,1.213220e+08,0.000496,1.091225e+08,0.001573,3.754384e+07,0.000096,1.548091e+08,0.0,777700000.0,798.663589
4,79,289.147649,1.5,0.365490,291.637255,1.0,0.962351,325.954103,1.0,0.748218,205.658342,1.5,0.855805,140.911256,10.0,0.170561,2.5,2.5,4.5,0,0,0,0,1,0,1,1,0,0,1,1,1,0,0,4,8,8,6,0,6,2,1,6,9,1,2,10,9,1,8,596,policy 155,dikesnet,2.800699e+08,0.0,1.184274e+08,0.000000,1.041480e+08,0.000000,3.644963e+07,0.000000,1.548091e+08,0.0,777700000.0,0.000000
5,28,277.745720,1.5,0.762077,253.886677,1.5,0.937489,307.076287,1.5,0.034439,94.152109,1.5,0.382453,336.135945,1.0,0.447764,4.5,1.5,4.5,0,0,0,0,1,0,1,1,0,0,1,1,1,0,0,4,8,8,6,0,6,2,1,6,9,1,2,10,9,1,8,597,policy 155,dikesnet,2.800699e+08,0.0,1.186594e+08,0.000041,2.318620e+08,0.036907,3.644963e+07,0.000000,1.548091e+08,0.0,777700000.0,11431.375403
6,108,199.155453,10.0,0.198897,341.337232,10.0,0.513497,67.807572,1.5,0.953576,164.513449,10.0,0.981957,267.217599,1.0,0.637847,3.5,3.5,4.5,0,0,0,0,1,0,1,1,0,0,1,1,1,0,0,4,8,8,6,0,6,2,1,6,9,1,2,10,9,1,8,598,policy 155,dikesnet,2.800699e+08,0.0,1.239725e+08,0.000766,1.041480e+08,0.000000,3.644963e+07,0.000000,1.548091e+08,0.0,777700000.0,430.058895
7,71,318.666881,1.0,0.044078,150.545208,1.5,0.267865,51.514572,10.0,0.594364,131.023201,1.0,0.225274,178.535387,10.0,0.120113,3.5,3.5,3.5,0,0,0,0,1,0,1,1,0,0,1,1,1,0,0,4,8,8,6,0,6,2,1,6,9,1,2,10,9,1,8,599,policy 155,dikesnet,2.800699e+08,0.0,1.442719e+08,0.003432,1.041480e+08,0.000000,3.722143e+07,0.000058,1.548091e+08,0.0,777700000.0,2112.441919
8,20,73.866337,1.5,0.568586,115.713279,1.0,0.572757,108.054842,10.0,0.892227,343.365131,1.0,0.555589,154.890071,1.0,0.836435,4.5,2.5,1.5,0,0,0,0,1,0,1,1,0,0,1,1,1,0,0,4,8,8,6,0,6,2,1,6,9,1,2,10,9,1,8,600,policy 155,dikesnet,2.800699e+08,0.0,1.230084e+08,0.000777,1.041480e+08,0.0